In [13]:
# เก็บข้อมูลจากเว็บ

import pandas as pd
from selenium import webdriver


def get_set50():
    driver = webdriver.Chrome()
    driver.get('https://www.set.or.th/th/market/index/set50/overview')
    return driver


driver = get_set50()


def get_set_df(driver):
    data = driver.page_source
    data_df = pd.read_html(data)[2]
    clean_column = []
    for c in data_df.columns:
        clean_column.append(c.replace('  (Click to sort Ascending)', ''))
    data_df.columns = clean_column
    data_df = data_df.set_index('หลักทรัพย์')
    return data_df


data_df = get_set_df(driver)


def get_stock_data(stock, driver, all_stock_dict):
    stock = stock.split()[0]
    url = 'https://www.set.or.th/th/market/product/stock/quote/' + \
        stock+'/financial-statement/company-highlights'
    driver.get(url)
    stock_data = driver.page_source
    a_df = pd.read_html(stock_data)[1]
    all_stock_dict[stock] = a_df
    return driver, all_stock_dict


all_stock_dict = dict()
for stock in data_df.index:
    driver, all_stock_dict = get_stock_data(stock, driver, all_stock_dict)


In [14]:
# ล็อคตารางที่ต้องการ

for key in all_stock_dict.keys():
    stock_df = all_stock_dict[key]
    stock_df.set_index('ค่าสถิติสำคัญ ณ วันที่', inplace=True)

In [15]:
# ฟังก์ชันตาราง

result_dict = dict()


def update_select_data(key, select_column, select_row, result_dict):
    try:
        select_data = all_stock_dict[key][select_column][select_row]
        result_dict[key] = float(select_data)
    except KeyError:
        pass
    except ValueError:
        pass
    return result_dict


def generate_df(result_dict, select_row):
    result_df = pd.DataFrame([result_dict])
    result_df = result_df.transpose()
    result_df.columns = [select_row]

    return result_df


In [16]:
# Config

select_column = []

for i in range(0, len(all_stock_dict[key].columns)):
    select_column.append(str(all_stock_dict[key].columns[i]))

select_row = 'ราคาล่าสุด (บาท)'


In [17]:
# สร้างตาราง

priceDay_df = []

for i in range(0, len(select_column)):
    for key in all_stock_dict.keys():
        result_dict = update_select_data(
            key, select_column[i], select_row, result_dict)

    priceDay_df.append(generate_df(result_dict, select_column[i]))

resultPrice_df = priceDay_df[0]

for i in range(0, len(select_column)):
    resultPrice_df[select_column[i]] = priceDay_df[i]

resultPrice_df


,28 ธ.ค. 2561,30 ธ.ค. 2562,30 ธ.ค. 2563,30 ธ.ค. 2564,01 ก.ย. 2565
ADVANC,172.50,213.00,176.00,230.00,190.50
AOT,64.25,74.25,62.25,61.00,72.00
BANPU,14.80,11.90,11.00,10.60,14.00
BBL,203.00,160.00,118.50,121.50,136.00
BDMS,24.80,26.00,20.80,23.00,29.00
BEM,9.70,10.90,8.30,8.45,8.70
BGRIM,26.50,52.50,48.50,40.50,36.75
BH,187.50,147.00,120.00,141.00,216.00
BLA,26.25,20.70,21.00,37.25,35.50
BTS,9.55,13.20,9.30,9.35,8.30


In [18]:
# Jupyter to Excel

!jupyter nbconvert --to xls SET20Price.ipynb

[NbConvertApp] Converting notebook SET20Price.ipynb to xls
[NbConvertApp] Writing 7135 bytes to SET20Price.xlsx
